In [1]:
import math
import pygame
import random
import numpy as np
import os

WIDTH = 800
HEIGHT = 600
FPS = 50
tau = .02 #time that passes between frames
score1 = 0
score2 = 0
player1 = 0
player2 = 0
maxScore = 1
time = 0

pygame.init()
pygame.mixer.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong Game")
pygame.mouse.set_visible(0)
font = pygame.font.Font(None, 30)
bigfont = pygame.font.Font(None, 72)
clock = pygame.time.Clock()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class ball(pygame.sprite.Sprite):
    # ball constructor
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([10, 10])
        pygame.draw.circle(self.image, (255, 255, 255), (5,5), 5)
        self.rect = self.image.get_rect()
        self.screenheight = HEIGHT
        self.screenwidth = WIDTH
        self.speed = 8
        self.x_pos = WIDTH/2
        self.y_pos = HEIGHT/2
        self.x_vel = 0
        self.y_vel = 0
        self.maxSpeed = 25
        self.velocity = np.array([0, 0])
        self.width = 10
        self.height = 10
        self.reset()

    def reset(self):
        self.rect.center = [WIDTH / 2, HEIGHT / 2]
        self.speed = 8.0
        self.x_vel = random.uniform(-1, 1)
        self.y_vel = random.uniform(-1, 1)
        self.velocity = np.array([self.x_vel / math.sqrt((self.x_vel ** 2)+(self.y_vel ** 2)),
                                  self.y_vel/math.sqrt((self.x_vel ** 2)+(self.y_vel ** 2))])  # <== [VelX, VelY]
        global time
        time = 0

    def paddleBounce(self, diff):
        self.rect.centerx -= self.speed * self.velocity[0]
        self.velocity[0] = -1 * self.velocity[0]
        self.velocity[1] = -diff/45
        if self.speed < self.maxSpeed:
            self.speed += .2
        self.velocity = np.array([(self.velocity[0] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2))),
                                  (self.velocity[1] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2)))])

    def sideWallBounce(self):
        self.velocity[0] = -1 * self.velocity[0]
        self.velocity[1] = self.velocity[1] * (random.random() + 1)
        self.velocity = np.array([(self.velocity[0] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2))),
                                  (self.velocity[1] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2)))])

    def TopAndBottomBounce(self):
        self.velocity[0] = self.velocity[0] * (random.random() + 1)
        self.velocity[1] = -1.1 * self.velocity[1]
        self.velocity = np.array([(self.velocity[0] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2))),
                                  (self.velocity[1] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2)))])

    def player1goal(self):
        global score1
        score1 += 1
        self.reset()

    def player2goal(self):
        global score2
        score2 += 1
        self.reset()


    def update(self):
        if time >= 50:              # provides a short delay before the ball starts to move
            self.rect.center += self.speed * self.velocity
            if self.rect.top < 1:
                self.velocity[1] = -.8 # * self.velocity[1]
                self.rect.centery = 7
            if self.rect.bottom > HEIGHT-0:
                self.velocity[1] = -1.1 * self.velocity[1]
                self.rect.bottom = HEIGHT-5
            if self.rect.left < 1 and self.rect.top < 160:
                self.rect.left = 3
                self.velocity[0] = 1.1 * self.velocity[0]
            if self.rect.left < 1 and self.rect.bottom > HEIGHT-160:
                self.rect.left = 3
                self.velocity[0] = 1.1 * self.velocity[0]
            if self.rect.right > WIDTH-1 and self.rect.top < 160:
                self.rect.right = WIDTH-3
                self.velocity[0] = 1.1 * self.velocity[0]
            if self.rect.right > WIDTH-1 and self.rect.bottom > HEIGHT-160:
                self.rect.right = WIDTH-3
                self.velocity[0] = 1.1 * self.velocity[0]
            self.velocity = np.array([(self.velocity[0] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2))),
                                    (self.velocity[1] / math.sqrt((self.velocity[0] ** 2) + (self.velocity[1] ** 2)))])
            
            
class paddle(pygame.sprite.Sprite):
    # paddle constructor
    def __init__(self, x_pos, upKey, downKey, vel):
        super().__init__()
        self.width = 10
        self.height = 90
        self.image = pygame.Surface([self.width, self.height])
        self.image.fill((255, 0, 0))
        self.speed = 0
        self.up = upKey
        self.down = downKey
        self.vel = vel
        self.rect = self.image.get_rect()
        self.screenheight = HEIGHT
        self.screenwidth = WIDTH

        self.rect.centery = HEIGHT / 2
        self.rect.centerx = x_pos

    # paddle move function
    def update(self):

        self.speed = 0
        keys = pygame.key.get_pressed()
        if keys[self.up]:
            self.speed = -1 * self.vel
        if keys[self.down]:
            self.speed = self.vel
        self.rect.y += self.speed
        if self.rect.top < 0:
            self.rect.top = 0
        if self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT
                        
class computerPaddle(pygame.sprite.Sprite):
    # computerPaddle constructor
    def __init__(self, x_pos, vel):
        super().__init__()
        self.width = 10
        self.height = 90
        self.image = pygame.Surface([self.width, self.height])
        self.image.fill((255, 0, 0))
        self.speed = 0
        self.vel = vel
        self.rect = self.image.get_rect()
        self.screenheight = HEIGHT
        self.screenwidth = WIDTH

        self.rect.centery = HEIGHT / 2
        self.rect.centerx = x_pos

    # paddle move function
    def update(self, action):

        self.speed = 0
        
        if action = 1:
            self.speed = -1 * self.vel
        if action = 0:
            self.speed = 0
        if action = -1:    
            self.speed = self.vel
        self.rect.y += self.speed
        if self.rect.top < 0:
            self.rect.top = 0
        if self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT
                        
            
class sideWall(pygame.sprite.Sprite):
    def __init__(self, x_pos, height, y_pos):
        super().__init__()
        self.width = 5
        self.height = height
        self.image = pygame.Surface([self.width, self.height])
        self.image.fill((255, 255, 0))
        self.rect = self.image.get_rect()
        self.screenheight = HEIGHT
        self.screenwidth = WIDTH
        self.rect.centery = y_pos
        self.rect.x = x_pos
        
class TandBWall(pygame.sprite.Sprite):
    def __init__(self, y_pos):
        super().__init__()
        self.width = WIDTH
        self.height = 5
        self.image = pygame.Surface([self.width, self.height])
        self.image.fill((255, 255, 0))
        self.rect = self.image.get_rect()
        self.screenheight = HEIGHT
        self.screenwidth = WIDTH
        self.rect.centerx = WIDTH / 2
        self.rect.y = y_pos
        



In [3]:
# Create groups and sprites



all_sprites = pygame.sprite.Group()
paddles = pygame.sprite.Group()
balls = pygame.sprite.Group()
sideWalls = pygame.sprite.Group()
TandBWalls = pygame.sprite.Group()

paddle1 = computerPaddle(50, 5)                                  # make left paddle
#all_sprites.add(paddle1)
paddles.add(paddle1)

paddle2 = paddle((WIDTH-50), pygame.K_UP, pygame.K_DOWN, 5)      # make right paddle
all_sprites.add(paddle2)
#paddles.add(paddle2)

ball1 = ball()                                                   # make ball
all_sprites.add(ball1)
balls.add(ball1)

TleftWall = sideWall(-1, 160, 80)                                # make top left wall
sideWalls.add(TleftWall)

TrightWall = sideWall(WIDTH-4, 160, 80)                          # make top right wall
sideWalls.add(TrightWall)

BleftWall = sideWall(-1, 160, HEIGHT-80)                         # make bottom left wall
sideWalls.add(BleftWall)

BrightWall = sideWall(WIDTH-4, 160, HEIGHT-80)                   # make bottom right wall
sideWalls.add(BrightWall)

TopWall = TandBWall(-1)                                          # make top wall
TandBWalls.add(TopWall)

BottomWall = TandBWall(HEIGHT-4)                                 # make bottom wall
TandBWalls.add(BottomWall)


running = True
start = False
done = False
time = 0

while running:
    clock.tick(FPS)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    if start:
        startText = "Welcome to Pong Project!"
        text = bigfont.render(startText, 1, (255, 255, 255))
        textPos = (85, 90)
        screen.blit(text, textPos)
        pygame.display.flip()

        startText = "Player 1 uses 'W' and 'S' to move the left paddle up and down"
        text = font.render(startText, 0, (255, 255, 255))
        textPos = (100, 200)
        screen.blit(text, textPos)
        pygame.display.flip()

        startText = "Player 2 uses the UP and DOWN arrow keys"
        text = font.render(startText, 0, (255, 255, 255))
        textpos = (180, 230)
        screen.blit(text, textpos)
        pygame.display.flip()

        startText = "to move the right paddle up and down"
        text = font.render(startText, 0, (255, 255, 255))
        textpos = (210, 260)
        screen.blit(text, textpos)
        pygame.display.flip()

        startText = "The first player to reach " + str(maxScore) + " point(s) is the winner"
        text = font.render(startText, 0, (255, 255, 255))
        textpos = (165, 290)
        screen.blit(text, textpos)
        pygame.display.flip()

        startText = "Press 'Q' at any point to quit the game"
        text = font.render(startText, 0, (255, 255, 255))
        textPos = (210, 320)
        screen.blit(text, textPos)
        pygame.display.flip()

        startText = "Press one of the movement keys when ready to play"
        text = font.render(startText, 0, (255, 255, 255))
        textPos = (140, 350)
        screen.blit(text, textPos)
        pygame.display.flip()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_UP]:
            start = False
        if keys[pygame.K_DOWN]:
            start = False
        if keys[pygame.K_w]:
            start = False
        if keys[pygame.K_s]:
            start = False

    if not start and not done:

        all_sprites.update()
        action = agent.act(state)
        paddles.update(action)
        

        if pygame.sprite.spritecollide(paddle1, balls, False, False):
            diff = (paddle1.rect.y + paddle1.height / 2) - (ball1.rect.y + ball1.height / 2)
            ball1.paddleBounce(diff)

        if pygame.sprite.spritecollide(paddle2, balls, False, False):
            diff = (paddle2.rect.y + paddle2.height / 2) - (ball1.rect.y + ball1.height / 2)
            ball1.paddleBounce(diff)

        sideWallBounce = pygame.sprite.groupcollide(balls, sideWalls, False, False)      # tests for side wall bounces
        if sideWallBounce:
            ball1.sideWallBounce()

        TopAndBottomBounce = pygame.sprite.groupcollide(balls, TandBWalls, False, False)  # tests for T and B wall bounces
        if TopAndBottomBounce:
            ball1.TopAndBottomBounce()

        # player1 goal
        if ball1.rect.centerx > WIDTH + 5:
            ball1.player1goal()

        # player2 goal
        if ball1.rect.centerx < -5:
            ball1.player2goal()

        if score1 == maxScore:
            player1 = 1
            done = True

        if score2 == maxScore:
            player2 = 1
            done = True

        screen.fill((0, 0, 0))

        scorePrint = "Player 1   " + str(score1) + " : " + str(maxScore)
        text = font.render(scorePrint, 1, (255, 255, 255))
        textPos = (180, 70)
        screen.blit(text, textPos)

        scorePrint = "Player 2   " + str(score2) + " : " + str(maxScore)
        text = font.render(scorePrint, 1, (255, 255, 255))
        textPos = (480, 70)
        screen.blit(text, textPos)

        all_sprites.draw(screen)
        paddles.draw(screen)
        sideWalls.draw(screen)
        TandBWalls.draw(screen)
        time += 1
        pygame.display.flip()

    if done:
        finalText = "GAME OVER!"
        text = bigfont.render(finalText, 1, (255, 255, 255))
        textPos = (245, 155)
        screen.blit(text, textPos)
        pygame.display.flip()
        if player1 == 1:
            winnerText = "Player 1 is the winner!!!"
            text = bigfont.render(winnerText, 1, (255, 255, 255))
            textPos = (120, 250)
            screen.blit(text, textPos)
            pygame.display.flip()
        if player2 == 1:
            winnerText = "Player 2 is the winner!!!"
            text = bigfont.render(winnerText, 1, (255, 255, 255))
            textPos = (120, 250)
            screen.blit(text, textPos)
            pygame.display.flip()
        finalText = "If you would like play again, press 'ENTER'"
        text = font.render(finalText, 0, (255, 255, 255))
        textPos = (195, 360)
        screen.blit(text, textPos)
        pygame.display.flip()

        finalText = "Otherwise press 'Q' to quit the game"
        text = font.render(finalText, 0, (255, 255, 255))
        textPos = (225, 400)
        screen.blit(text, textPos)
        pygame.display.flip()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_RETURN]:
            player1 = 0
            player2 = 0
            score1 = 0
            score2 = 0
            paddle1.rect.centery = HEIGHT / 2
            paddle2.rect.centery = HEIGHT / 2
            done = False
    keys = pygame.key.get_pressed()
    if keys[pygame.K_q]:
        running = False

pygame.quit()